# https://www.kaggle.com/c/digit-recognizer/

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms

# dataset

In [5]:
batch_size = 1
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,), (0.3081,))
                                             ])),
        batch_size=batch_size, shuffle=True)

print(len(train_loader))

60000


# model

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# train

In [4]:
model = CNN().cuda()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)

for epoch in range(1, 100 + 1):
    # train
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))
    
    # evaluate
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
    
torch.save(model.state_dict(),"mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299318
Test set: Average loss: 1.3503, Accuracy: 7682/10000 (77%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.349840
Test set: Average loss: 0.4465, Accuracy: 8839/10000 (88%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.368421
Test set: Average loss: 0.3227, Accuracy: 9095/10000 (91%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.353859
Test set: Average loss: 0.2708, Accuracy: 9210/10000 (92%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.295237
Test set: Average loss: 0.2338, Accuracy: 9318/10000 (93%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.251779
Test set: Average loss: 0.2065, Accuracy: 9408/10000 (94%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.246128
Test set: Average loss: 0.1861, Accuracy: 9467/10000 (95%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.140893
Test set: Average loss: 0.1697, Accuracy: 9505/10000 (95%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.107806
Test set: Average loss: 0.1538, Accuracy: 9561/10000 (96%)

Train Epoch: 10 [0/60000 (0%)]	Loss: 0.172858
Test set:

In [5]:
!ls

data  mnist_cnn.pt  mnist.ipynb
